In [1]:
import graphlab as gl
import graphlab.aggregate as agg
import pandas as pd
import numpy as np
#import scipy
#import matplotlib.pyplot as plt
#import seaborn as sns
import math as math
import pyodbc
#import pysftp

In [2]:
# Create the connection to the sql server w.r.t to the DNS set
conn = pyodbc.connect("DSN=sq02", autocommit=True)

# SQL query across the tables
sql = """

-- dataB2B  RP
SELECT DISTINCT F.CustomerKey AS customerKey, C.Name AS customerName, I.[Vendor Item No_] AS vendorItemNo,
  I.[Vendor No_] AS vendorNo, I.ItemKey as itemKey, I.Description AS itemDescription, I.marchio, I.codMarchio,
  I.sport, I.reparto, I.sesso, I.tipo
  FROM [3A_DWH].[dbo].[FactSalesOrder] AS F
  JOIN [3A_DWH].[dbo].[DimItem] AS I
  ON I.ItemKey = F.ItemKey
  JOIN [3A_DWH].[dbo].[DimCustomer] AS C
  ON C.CustomerKey = F.CustomerKey
  join [3A_DWH].[dbo].[FactSalesConditions] as fsc
  on C.CustomerKey = fsc.CustomerKey and
  i.codMarchio = fsc.[Brand Code]
  WHERE C.Name NOT LIKE 'T%' AND YEAR(F.[Order Date]) > 2016 AND I.Company = '3A dei F_lli Antonini S_p_a_' and
  F.[BusinessType Code]='B2B'
  -- example
  --and C.CustomerKey like '%c06423%'
  ORDER BY F.CustomerKey, I.[ItemKey];
  
"""

In [3]:
# Read sql query output to panda dataframe
df = pd.io.sql.read_sql(sql, conn)
# Convert Dataframe to SFrame
dataB2B = gl.SFrame(df)

This non-commercial license of GraphLab Create for academic use is assigned to festo.owiny@studenti.unitn.it and will expire on August 10, 2019.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\FESTO~1.OWI\AppData\Local\Temp\2\graphlab_server_1540281033.log.0


In [5]:
# Save the sframe data and print the result for visualization
dataB2B.save('./dataset/dataB2B_RP.csv', format='csv')
#print dataB2B.num_rows(), '\n', dataB2B.column_names(), '\n', dataB2B.head(), '\n', df.head()

### Read dataset output by recommender_AAP.py

In [6]:
# Read data saved by the previous recommender_AAP.py run
dataB2B_AAP = gl.SFrame.read_csv('./dataset/dataB2B_AAP.csv')

Finished parsing file C:\Users\festo.owiny\Desktop\recommend\dataset\dataB2B_AAP.csv

Parsing completed. Parsed 100 lines in 0.058593 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str,str,str,str,str,str,str,str,str,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file C:\Users\festo.owiny\Desktop\recommend\dataset\dataB2B_AAP.csv

Parsing completed. Parsed 1137 lines in 0.046008 secs.

In [7]:
########## Derive dictionaries from item/product
dicCustomer = dataB2B.select_columns(['customerKey', 'customerName']).unique()
dicItem = dataB2B.select_columns(['itemKey', 'vendorItemNo', 'vendorNo', 'itemDescription', 'marchio', 'codMarchio',\
                                  'sport', 'reparto', 'sesso', 'tipo']).unique()
#### Count unique users/Products
users = dataB2B['customerKey'].unique()
products = dataB2B['itemKey'].unique()

In [8]:
# Derive dictionary for items/products of AAP
dicCustomer_AAP = dataB2B_AAP.select_columns(['customerKey', 'customerName']).unique()
dicItem_AAP = dataB2B_AAP.select_columns(['itemKey', 'vendorItemNo', 'vendorNo', 'itemDescription', 'marchio', 'codMarchio',\
                                          'sport', 'reparto', 'sesso', 'tipo']).unique()
#### Unique users/Products for AAP
users_AAP = dataB2B_AAP['customerKey'].unique()
products_AAP = dataB2B_AAP['itemKey'].unique()

### split dataset

In [9]:
## Split dataset into train and test ####
train_data, test_data = dataB2B.random_split(0.8, seed=0)

### POPULARITY MODEL ###

#### 1. Deriving full list of APP clients #####

In [10]:
########## Popularity-based recommender
model_pop = gl.popularity_recommender.create(dataB2B, user_id='customerKey', item_id='itemKey')
########## Recommned product for users using Popularity-based recommender
recommended_pop = model_pop.recommend(users, k=len(products))
recommended_pop = recommended_pop.join(dicCustomer, on = 'customerKey', how = 'inner').join(dicItem, on = 'itemKey', how = 'inner')

Recsys training: model = popularity

Warning: Ignoring columns customerName, vendorItemNo, vendorNo, itemDescription, marchio, codMarchio, sport, reparto, sesso, tipo;

To use one of these as a target column, set target = <column_name>

and use a method that allows the use of a target.

Preparing data set.

Data has 14784 observations with 923 users and 3201 items.

Data prepared in: 0.087014s

14784 observations to process; with 3201 unique items.

In [11]:
# AAP Full list (clients belong to both AAp and RP) with recommded items
popRP_N04 = recommended_pop.join(dicCustomer_AAP, on = 'customerKey', how = 'inner')
# Exclude N04 products as they already have in the previous run of recommender_AAP.py ... This step distorts the the flow of rank column
popRP1_N04 = popRP_N04[popRP_N04['codMarchio']!='N04']

In [12]:
## Deriving a new rank column to replace the current distorted ###
# Sort the results in ascending order of customerKey and rank respectively for later use in merging new rank column
popRP2_N04 = popRP1_N04.sort(['customerKey', 'rank'], ascending = True)
# Deriving the new rank column and then sort the numbers by customerKey and rankCount respectively to match  popRP2_N04 
extract = popRP2_N04.groupby('customerKey', {'rankCount': agg.COUNT('rank')}).sort(['customerKey', 'rankCount'], ascending = True)
# New rank as a lis of lists
rank1 = extract['rankCount'].apply(lambda x: np.arange(1, x+1))

# collapsing the list of lists
merged_list = []
for i in rank1:
    merged_list += i
# Casting the 'float' to 'int' to obtain a new list ready for merging
merged_list = map(int, merged_list)

In [13]:
# Append merged_list as a new column 'rank1'
popRP2_N04['rank1'] = merged_list
# Swap the rank columns
popRP2_N04.swap_columns('rank', 'rank1')
# Delete the last two columns and rename rank1 to rank
popRP2_N04.remove_columns(['customerName.1', 'rank']).rename({'rank1':'rank'})
# Obtain desired output by excluding values with rank greater than 10
popRP_NIKE = popRP2_N04[popRP2_N04['rank'] <= 10]

#### 2. Secondly derive RP_only List excluding the AAP clients above  ######

In [14]:
# RP List excluding the AAP clients
popRPonly = recommended_pop.filter_by(dicCustomer_AAP['customerKey'], 'customerKey', exclude=True)
popRPonly_list = popRPonly[popRPonly['rank'] <= 10]

### 3. Merge the two recommendation lists into a single list  ##########

In [15]:
popularRecommend_RP = popRP_NIKE.append(popRPonly_list)
print len(popRP_NIKE), len(popRPonly_list), len(popularRecommend_RP)

740 8490 9230


### PERSONALIZED RECOMMENDER ####

In [16]:
########## Personalized recommender
model_pers = gl.item_similarity_recommender.create(dataB2B, user_id='customerKey', item_id='itemKey')
########## Recommned product for users using Popularity-based recommender
recommended_pers = model_pers.recommend(users, k=len(products))
recommended_pers = recommended_pers.join(dicCustomer, on = 'customerKey', how = 'inner').join(dicItem, on = 'itemKey', how = 'inner')

Recsys training: model = item_similarity

Warning: Ignoring columns customerName, vendorItemNo, vendorNo, itemDescription, marchio, codMarchio, sport, reparto, sesso, tipo;

To use one of these as a target column, set target = <column_name>

and use a method that allows the use of a target.

Preparing data set.

...

| 193.011ms                           | 100              | 3201            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 1.20984s

#### 1. Deriving full list of APP clients #####

In [17]:
# AAP Full list (clients belong to both AAp and RP) with recommded items
persRP_N04 = recommended_pers.join(dicCustomer_AAP, on = 'customerKey', how = 'inner')
# Exclude N04 products as they already have in the previous run of recommender_AAP.py ... This step distorts the the flow of rank column
persRP1_N04 = persRP_N04[persRP_N04['codMarchio']!='N04']

In [18]:
## Deriving a new rank column to replace the current distorted ###
# Sort the results in ascending order of customerKey and rank respectively for later use in merging new rank column
persRP2_N04 = persRP1_N04.sort(['customerKey', 'rank'], ascending = True)
# Deriving the new rank column and then sort the numbers by customerKey and rankCount respectively to match  popRP2_N04 
extract1 = persRP2_N04.groupby('customerKey', {'rankCount': agg.COUNT('rank')}).sort(['customerKey', 'rankCount'], ascending = True)
# New rank as a lis of lists
rank2 = extract1['rankCount'].apply(lambda x: np.arange(1, x+1))

# collapsing the list of lists
merged_list1 = []
for j in rank2:
    merged_list1 += j
# Casting the 'float' to 'int' to obtain a new list ready for merging
merged_list1 = map(int, merged_list1)

In [19]:
# Append merged_list as a new column 'rank1'
persRP2_N04['rank1'] = merged_list1
# Swap the rank columns
persRP2_N04.swap_columns('rank', 'rank1')
# Delete the last two columns and rename rank1 to rank
persRP2_N04.remove_columns(['customerName.1', 'rank']).rename({'rank1':'rank'})
# Obtain desired output by excluding values with rank greater than 10
persRP_NIKE = persRP2_N04[persRP2_N04['rank'] <= 10]

#### 2. Secondly derive RP_only List excluding the AAP clients above  ######

In [20]:
# RP only clients excluding the AAP clients
persRPonly = recommended_pers.filter_by(dicCustomer_AAP['customerKey'], 'customerKey', exclude=True)
persRPonly_list = persRPonly[persRPonly['rank'] <= 10]

#### 3. Merge the two recommendation lists into a single list  ##########

In [21]:
persRecommend_RP = persRP_NIKE.append(persRPonly_list)
print len(persRP_NIKE), len(persRPonly_list), len(persRecommend_RP)

740 8490 9230


### 4. Full List of Most Similar Items by purchases pairs  #####

In [22]:
similar_purchases = model_pers.get_similar_items(products)
similar_purchases = similar_purchases.join(dicItem, on = 'itemKey', how = 'inner').join(dicItem, on = {'similar':'itemKey'}, how = 'inner')
similar_purchases.head(10)

itemKey,similar,score,rank,codMarchio,itemDescription,marchio
N04-00011038-3A dei F_lliAntonini S_p_a_ ...,N04-00013553-3A dei F_lliAntonini S_p_a_ ...,1.0,1,N04,WMNS NIKE COURT ROYALE SE,NIKE
N04-00011038-3A dei F_lliAntonini S_p_a_ ...,N04-00001544-3A dei F_lliAntonini S_p_a_ ...,1.0,2,N04,WMNS NIKE COURT ROYALE SE,NIKE
N04-00011038-3A dei F_lliAntonini S_p_a_ ...,N04-00008851-3A dei F_lliAntonini S_p_a_ ...,1.0,3,N04,WMNS NIKE COURT ROYALE SE,NIKE
N04-00011038-3A dei F_lliAntonini S_p_a_ ...,N04-00011028-3A dei F_lliAntonini S_p_a_ ...,1.0,4,N04,WMNS NIKE COURT ROYALE SE,NIKE
N04-00011038-3A dei F_lliAntonini S_p_a_ ...,N04-00011190-3A dei F_lliAntonini S_p_a_ ...,1.0,5,N04,WMNS NIKE COURT ROYALE SE,NIKE
N04-00011038-3A dei F_lliAntonini S_p_a_ ...,N04-00011191-3A dei F_lliAntonini S_p_a_ ...,1.0,6,N04,WMNS NIKE COURT ROYALE SE,NIKE
N04-00011038-3A dei F_lliAntonini S_p_a_ ...,N04-00011594-3A dei F_lliAntonini S_p_a_ ...,1.0,7,N04,WMNS NIKE COURT ROYALE SE,NIKE
N04-00011038-3A dei F_lliAntonini S_p_a_ ...,N04-00011596-3A dei F_lliAntonini S_p_a_ ...,1.0,8,N04,WMNS NIKE COURT ROYALE SE,NIKE
N04-00011038-3A dei F_lliAntonini S_p_a_ ...,N04-00001547-3A dei F_lliAntonini S_p_a_ ...,0.5,9,N04,WMNS NIKE COURT ROYALE SE,NIKE
N04-00011038-3A dei F_lliAntonini S_p_a_ ...,N04-00013031-3A dei F_lliAntonini S_p_a_ ...,0.5,10,N04,WMNS NIKE COURT ROYALE SE,NIKE


### 5. Lastly save the results/dataset as required  #####

In [23]:
########## Export PERS_output as json/csv
#persRecommend_RP.export_json('./output/personalizedRecommend_RP.json', orient='records')
#persRecommend_RP.save('./output/personalizedRecommend_RP.csv', format='csv')

In [24]:
########## Export POP_output as json/csv
#ecommended_pop.export_json('./output/AAP/popularRecommend_AAP.json', orient='records')
#recommended_pop.save('./output/AAP/popularRecommend_AAP.csv', format='csv')

########## Export of most Similarly purchased product pairs as json/csv
#similar_purchases.export_json('./output/similarPurchases_AAP.json', orient='records')
#similar_purchases.save('./output/similarPurchases_AAP.csv', format='csv')

### 6. Verify results for the personalized model

In [25]:
result_AAP = gl.SFrame.read_csv('./output/personalizedRecommend_AAP.csv')
result_RP = gl.SFrame.read_csv('./output/personalizedRecommend_RP.csv')

Finished parsing file C:\Users\festo.owiny\Desktop\recommend\output\personalizedRecommend_AAP.csv

Parsing completed. Parsed 100 lines in 0.077006 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str,long,long,str,str,str,str,str,str,str,str,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file C:\Users\festo.owiny\Desktop\recommend\output\personalizedRecommend_AAP.csv

Parsing completed. Parsed 740 lines in 0.053993 secs.

Finished parsing file C:\Users\festo.owiny\Desktop\recommend\output\personalizedRecommend_RP.csv

Parsing completed. Parsed 100 lines in 0.218207 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str,float,long,str,str,str,str,str,str,str,str,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file C:\Users\festo.owiny\Desktop\recommend\output\personalizedRecommend_RP.csv

Parsing completed. Parsed 9230 lines in 0.153396 secs.

In [26]:
vAAP = persRecommend_RP[persRecommend_RP['codMarchio']=='N04']

In [27]:
print len(result_AAP), len(result_RP), len(vAAP)

740 9230 6064


In [28]:
r1 = result_AAP[result_AAP['customerKey']=='C06423-3A dei F_lli Antonini S_p_a_']['codMarchio'].unique()
p1 = result_RP[result_RP['customerKey']=='C06423-3A dei F_lli Antonini S_p_a_']['codMarchio'].unique()
print r1, p1

['N04'] ['J11']


In [29]:
r2 = result_AAP[result_AAP['customerKey']=='C08099-3A dei F_lli Antonini S_p_a_']['codMarchio'].unique()
p2 = result_RP[result_RP['customerKey']=='C08099-3A dei F_lli Antonini S_p_a_']['codMarchio'].unique()
print r2, p2

['N04'] ['J11', 'C49']


In [30]:
r3 = result_AAP[result_AAP['customerKey']=='C07033-3A dei F_lli Antonini S_p_a_']['codMarchio'].unique()
p3 = result_RP[result_RP['customerKey']=='C07033-3A dei F_lli Antonini S_p_a_']['codMarchio'].unique()
print r3, p3

['N04'] ['J11']


In [31]:
r4 = result_AAP[result_AAP['customerKey']=='C04708-3A dei F_lli Antonini S_p_a_']['codMarchio'].unique()
p4 = result_RP[result_RP['customerKey']=='C04708-3A dei F_lli Antonini S_p_a_']['codMarchio'].unique()
print r4, p4

['N04'] ['J11']


In [32]:
persRP_N04['customerKey'].unique()[30]

'C04708-3A dei F_lli Antonini S_p_a_'

In [62]:
#result_RP[result_RP['codMarchio'] != 'N04'  result_RP['codMarchio'] != 'J11'].unique()

In [38]:
result_AAP.head(5)

Columns:
	customerKey	str
	itemKey	str
	score	int
	rank	int
	customerName	str
	codMarchio	str
	itemDescription	str
	marchio	str
	reparto	str
	sesso	str
	sport	str
	tipo	str
	vendorItemNo	str
	vendorNo	str

Rows: 5

Data:
+-------------------------------+-------------------------------+-------+------+
|          customerKey          |            itemKey            | score | rank |
+-------------------------------+-------------------------------+-------+------+
| C06981-3A dei F_lli Antoni... | N04-00009233-3A dei F_lli ... |   7   |  7   |
| C07121-3A dei F_lli Antoni... | N04-00001024-3A dei F_lli ... |   7   |  9   |
| C06981-3A dei F_lli Antoni... | N04-00001024-3A dei F_lli ... |   7   |  8   |
| C07146-3A dei F_lli Antoni... | N04-00002835-3A dei F_lli ... |   9   |  8   |
| C03782-3A dei F_lli Antoni... | N04-00002835-3A dei F_lli ... |   9   |  9   |
+-------------------------------+-------------------------------+-------+------+
+-------------------------------+------------+-----------------------------+
|          customerName         | codMarchio |       itemDescription       |
+-------------------------------+------------+-----------------------------+
| CENTER SPORT DI PASCOLINI ... |    N04     |     NIKE REVOLUTION 4 EU    |
|         AREA SPORT SRL        |    N04     |  NIKE TEAM HUSTLE D 8 (GS)  |
| CENTER SPORT DI PASCOLINI ... |    N04     |  NIKE TEAM HUSTLE D 8 (GS)  |
|       SPORT SERVICE SRL       |    N04     | 3PPK VALUE COTTON CREW-SMLX |
|        VACANZE 2000 SRL       |    N04     | 3PPK VALUE COTTON CREW-SMLX |
+-------------------------------+------------+-----------------------------+
+---------+-----------+--------+------------------------+--------+--------------+
| marchio |  reparto  | sesso  |         sport          |  tipo  | vendorItemNo |
+---------+-----------+--------+------------------------+--------+--------------+
|   NIKE  | CALZATURE |  UOMO  |     RUNNING/CORSA      | SCARPE |  AJ3490-001  |
|   NIKE  | CALZATURE | JUNIOR | TEMPO LIBERO PRIMAVERA | SCARPE |  881941-100  |
|   NIKE  | CALZATURE | JUNIOR | TEMPO LIBERO PRIMAVERA | SCARPE |  881941-100  |
|   NIKE  | ACCESSORI |  UOMO  |  FITNESS/ALLENAMENTO   | CALZE  |  SX4508-101  |
|   NIKE  | ACCESSORI |  UOMO  |  FITNESS/ALLENAMENTO   | CALZE  |  SX4508-101  |
+---------+-----------+--------+------------------------+--------+--------------+
+----------+
| vendorNo |
+----------+
|  F03269  |
|  F03269  |
|  F03269  |
|  F03269  |
|  F03269  |
+----------+
[5 rows x 14 columns]

In [37]:
result_RP.head(5)

Columns:
	customerKey	str
	itemKey	str
	score	float
	rank	int
	customerName	str
	codMarchio	str
	itemDescription	str
	marchio	str
	reparto	str
	sesso	str
	sport	str
	tipo	str
	vendorItemNo	str
	vendorNo	str

Rows: 5

Data:
+-------------------------------+-------------------------------+-----------+------+
|          customerKey          |            itemKey            |   score   | rank |
+-------------------------------+-------------------------------+-----------+------+
| C02282-3A dei F_lli Antoni... | NSW-00000054-3A dei F_lli ... | 0.0333333 |  1   |
| C02282-3A dei F_lli Antoni... | NSW-00000108-3A dei F_lli ... |  0.031746 |  2   |
| C02282-3A dei F_lli Antoni... | N29-00001331-3A dei F_lli ... |    0.0    |  3   |
| C02282-3A dei F_lli Antoni... | N29-00001783-3A dei F_lli ... |    0.0    |  4   |
| C02282-3A dei F_lli Antoni... | N29-00002924-3A dei F_lli ... |    0.0    |  5   |
+-------------------------------+-------------------------------+-----------+------+
+---------------------+------------+------------------------------+
|     customerName    | codMarchio |       itemDescription        |
+---------------------+------------+------------------------------+
| CILLO DI BONI ITALO |    NSW     |     RACERBACK ONE PIECE      |
| CILLO DI BONI ITALO |    NSW     |     RACERBACK ONE PIECE      |
| CILLO DI BONI ITALO |    C49     |    CTP SHINY SILICONE TEE    |
| CILLO DI BONI ITALO |    J11     |         SESSION PACK         |
| CILLO DI BONI ITALO |    J11     | AIR JORDAN RETRO 1 HIGH CREW |
+---------------------+------------+------------------------------+
+-----------------+---------------+--------+--------------+---------+
|     marchio     |    reparto    | sesso  |    sport     |   tipo  |
+-----------------+---------------+--------+--------------+---------+
|    NIKE SWIM    | ABBIGLIAMENTO | DONNA  | PISCINA/MARE | COSTUME |
|    NIKE SWIM    | ABBIGLIAMENTO | DONNA  | PISCINA/MARE | COSTUME |
| HADDAD CONVERSE | ABBIGLIAMENTO | JUNIOR |   FASHION    | T-SHIRT |
|  HADDAD JORDAN  | ABBIGLIAMENTO | JUNIOR |   FASHION    |  ZAINO  |
|  HADDAD JORDAN  | ABBIGLIAMENTO | JUNIOR |   FASHION    |  CALZE  |
+-----------------+---------------+--------+--------------+---------+
+--------------+----------+
| vendorItemNo | vendorNo |
+--------------+----------+
| NESS8074-001 |  F04335  |
| NESS8074-580 |  F04335  |
|  868096-N3C  |  F04151  |
|  9A1985-G3X  |  F04343  |
|  UJ0089-R78  |  F04343  |
+--------------+----------+
[5 rows x 14 columns]